In [1]:
import os
import cv2
import numpy as np
import time

In [3]:
def custom_filter_operation(image):

    Start_time = time.time()
    blurred_image = cv2.GaussianBlur(image, (3,3), 0)
    print(f"Time taken to apply Gaussian Blur to an image: {time.time() - Start_time}s")
        
    Start_time = time.time()
    # Compute the elevation map using Sobel operator for edge detection
    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
        
    # Combine the gradients to obtain the magnitude of the gradient
    magnitude_gradient = cv2.magnitude(sobel_x, sobel_y)
    print(f"Time taken to apply Sobel filter to an image: {time.time() - Start_time}s")
    
    Start_time = time.time()
    # Normalize the image to a range between 0 and 1 (floating-point operations)
    image = magnitude_gradient.astype(np.float32) / 255.0
    print(f"Time taken to normalize the 1 image: {time.time() - Start_time}s")

    # Get the dimensions of the image
    height, width = image.shape

    # Create an output image initialized to zero
    output_image = np.zeros((height, width), dtype=np.float32)

    # Custom filter (2D array) for processing
    filter_kernel = np.array([[0.1, 0.2, 0.1],
                              [0.2, 0.1, 0.2],
                              [0.1, 0.2, 0.1],], dtype=np.float32)
    filter_size = filter_kernel.shape[0]

    # Nested loops to apply the filter to the image
    Start_time = time.time()
    for i in range(height):
        for j in range(width):
            # Accumulate the weighted sum for the filter
            weighted_sum = 0.0
            for k in range(filter_size):
                for l in range(filter_size):
                    # Calculate the position in the original image
                    row = i + k - filter_size // 2
                    col = j + l - filter_size // 2
                    
                    # Conditional loop: Check if indices are within bounds
                    if 0 <= row < height and 0 <= col < width:
                        weighted_sum += image[row, col] * filter_kernel[k, l]

            # Set the pixel value in the output image (floating-point operation)
            output_image[i, j] = weighted_sum

    print(f"Time taken to apply filter to 1 image (Nested Loop): {time.time() - Start_time}s")

    Start_time = time.time()    
    # Clip the output to stay in the range [0, 1]
    output_image = np.clip(output_image, 0.0, 1.0)
    print(f"Time taken to Clip the output to stay in the range [0, 1]: {time.time() - Start_time}s")

    Start_time = time.time()
    # Convert the processed image back to 8-bit (0-255)
    processed_image = (output_image * 255).astype(np.uint8)
    print(f"Time taken to Convert the image back to 8-bit (0-255): {time.time() - Start_time}s")

    return processed_image

In [4]:
# Folder containing the images
input_folder = 'cars_sample'  # Replace with the path to your folder
output_folder = 'I:\\Parallel\\cars_filtered'  # Replace with the path to save filtered images

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [5]:
Start_time_final = time.time()
# Loop through all the files in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Check for image file types
        # Read the image in grayscale
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            print(f"Could not open or find the image: {image_path}")

        # Apply the custom filter operation
        Start_time = time.time()
        filtered_image = custom_filter_operation(image)
        print(f"Total Time taken to apply custom filter to the image: {time.time() - Start_time}s")

        # Save the processed image to the output folder
        output_path = os.path.join(output_folder, f'filtered_{filename}')
        cv2.imwrite(output_path, filtered_image)

        print(f'Processed and saved: {output_path}')
        print("\n")
        
print(f"Total time taken by the execution of this code sequentially for all images: {time.time() - Start_time_final}s")

Time taken to apply Gaussian Blur to an image: 0.0025000572204589844s
Time taken to apply Sobel filter to an image: 0.0019979476928710938s
Time taken to normalize the 1 image: 0.0010042190551757812s
Time taken to apply filter to 1 image (Nested Loop): 1.738670825958252s
Time taken to Clip the output to stay in the range [0, 1]: 0.0005002021789550781s
Time taken to Convert the image back to 8-bit (0-255): 0.0004999637603759766s
Total Time taken to apply custom filter to the image: 1.7451732158660889s
Processed and saved: I:\Parallel\cars_filtered\filtered_carsgraz_001.bmp


Time taken to apply Gaussian Blur to an image: 0.0s
Time taken to apply Sobel filter to an image: 0.0024995803833007812s
Time taken to normalize the 1 image: 0.0004994869232177734s
Time taken to apply filter to 1 image (Nested Loop): 1.7335562705993652s
Time taken to Clip the output to stay in the range [0, 1]: 0.0004999637603759766s
Time taken to Convert the image back to 8-bit (0-255): 0.0s
Total Time taken to appl